<a href="https://colab.research.google.com/github/vischia/adfm_2024-2025/blob/master/01_first_steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de Datos en Física Moderna 2024-2025
@Pietro Vischia (pietro.vischia@cern.ch)



This is a jupyter notebook.

You can execute the content of a cell by pressing `SHIFT+ENTER`.

Cells are of two types:
- "Markdown": cells like the one you are reading, containing some text or/and images to be displayed.
- "Code": cells that contain code that needs to be executed.

This particular notebook, because of its extension, is a python notebook, therefore the code will have to be in python.

In cells of type "Code", there are a few special characters that control how a line is run:
- By default, the line is passed through the python interpreter
- Lines (or portion of lines) starting with `#` are considered comments and are not executed
- Lines that start with `!` or `%` are considered shell commands, and are executed as if they were in a terminal (as if you were in the command line where you would normally e.g. run the command `python`).

If you are running this in Google Colab, you will need to set up some initial commands in order to be able to use your Google Drive to download data. The next cell is a "Code" cell, but it is all commented out. Follow the instructions to set things up.

In [ ]:
# If you are running on Colab:
# Uncomment and run the following lines (remove only the "#", keep the "!").
# You can run it anyway, but it will do nothing if you have already installed all dependencies
# (and it will take some time to tell you it is not gonna do anything)
#from google.colab import drive
#drive.mount('/content/drive')
#%cd "/content/drive/MyDrive/"
#! git clone https://github.com/vischia/adfm_2024-2025.git
#%cd adfm_2024-2025
#%pip install shap torchinfo livelossplot

# If you are not running on Colab:
# Uncomment and run the following lines, to see what happens
#!pwd
#!ls
#%pip install shap torchinfo livelossplot

### A simple dataset

Let's explore a simple dataset: houses in San Francisco and New York. The dataset is available through https://github.com/jadeyee/r2d3-part-1-data .

In [ ]:
# Fetch the data  
!wget https://raw.githubusercontent.com/jadeyee/r2d3-part-1-data/refs/heads/master/part_1_data.csv

### Load the data

Now the data are on the disk. We need to load them into a readable structure.

It is convenient to use the `pandas` library for this, because it loads the data into structures with handy column titles, and it has a lot of utilities to manipulate the data.



In [ ]:
import pandas as pd

data = pd.read_csv("./part_1_data.csv", skip_lines=2,sep=",")

The loading fails. Read the error message, then open the file with an editor to figure out what the issue may be and how to solve it.

Hint: you can use the option `skip_lines=...` of `pandas.read_csv()`

## Data inspection

The first thing you need to do when building a machine learning model is to forget about the model, and **just look at the data**

In [ ]:
data.head()

## Plotting histograms of some observables using matplotlib

(see also examples on [matplotlib](https://matplotlib.org/3.5.3/api/_as_gen/matplotlib.pyplot.html) website)

In [ ]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = (8, 6)
matplotlib.rcParams['axes.labelsize'] = 14

print(data.columns)
plt.hist(data["beds"], bins=100)
plt.xlabel("Number of beds")
plt.ylabel("Events")
plt.show()

In [ ]:
import seaborn as sns
sns.set()
cols_to_plot = [
    'Hreco_Lep1_pt',
    'Hreco_HadTop_pt',
    'Hreco_All5_Jets_pt',
    'Hreco_More5_Jets_pt',
    'Hreco_Jets_plus_Lep_pt',
    'label'
]
pp=sns.pairplot(data=data, hue='in_sf', diag_kws={'bw_method': 0.2})
pp.map_lower(sns.kdeplot, levels=4, color=".2") # Contours


### Split the data set into training and test set

When we train a machine learning algorithm, we are trying to solve an interpolation problem (*find the function of the input features that provides the best approximation of the true function*) by also requiring that the solution generalizes sufficiently well (*the interpolating function must also predict correctly the value of the true function for new, unseen data*).


When we have a labelled dataset, we will therefore split it into: a *training set*, which we will use to train the machine learning algorithm; a *test set*, which we will use to evaluate the performance of the algorithm for various realizations of the algorithm (e.g. tuning hyperparameters); and an *application set*, which are the data we are really interested in studying in the end.

For many applications, when the amount of hyperparameters tuning is moderate, application set and test set can be collapsed into a single set (usually called *test set*). This is what we will do in this tutorial.

![Blah](figs/trainingNetwork.png)

(Image: P. Vischia, [doi:10.5281/zenodo.6373442](https://doi.org/10.5281/zenodo.6373442))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.tree import export_graphviz
from sklearn.inspection import permutation_importance
try:
    # See #1137: this allows compatibility for scikit-learn >= 0.24
    from sklearn.utils import safe_indexing
except ImportError:
    from sklearn.utils import _safe_indexing
import numpy as np    
    
X = data.drop(["in_sf"], axis=1)
y = data["in_sf"]


import sklearn
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.33, random_state=42)

print(f"We have {len(X_train)} training samples with {sum(y_train)} signal and {sum(1-y_train)} background events")
print(f"We have {len(X_test)} testing samples with {sum(y_test)} signal and {sum(1-y_test)} background events")

Now we will choose a simple criterion, for instance the value of one of the features that characterize the houses, and use it to decide if the house is in New York (we want to predict `0`) or in San Francisco (we want to predict `1`)

In [ ]:
def predict(X, var, val):
    pred = X[var]
    return np.array([0 if p > val  else 1 for p in pred])

    
def score(myX, myY, var, val):
    return accuracy_score(myY, predict(myX, var, val))


Let's try a few of them

In [ ]:
tested_var="price"
threshold_val=100000 # dollars
print('Personalised score: train accuracy', score(X_train, y_train, tested_var, threshold_val), ', test accuracy', score(X_test, y_test, tested_var, threshold_val))
threshold_val=1000000 # dollars
print('Personalised score: train accuracy', score(X_train, y_train, tested_var, threshold_val), ', test accuracy', score(X_test, y_test, tested_var, threshold_val))



Now let's try out a "thresholding on steroids" machine learning classifier, called Boosted Decision Tree (BDT), and compare its accuracy with the personalised score one.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier


learning_rate = 1.0

bdt_ada = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=3, criterion='log_loss'), n_estimators=100, learning_rate=learning_rate, random_state=42)
fitted_bdt_ada=bdt_ada.fit(X_train, y_train)

tested_var="price"
threshold_val=1000000 # dollars

print('Adaptive boost: train accuracy', fitted_bdt_ada.score(X_train, y_train),', test accuracy', fitted_bdt_ada.score(X_test, y_test))
print('Personalised score: train accuracy', score(X_train, y_train, tested_var, threshold_val), ', test accuracy', score(X_test, y_test, tested_var, threshold_val))


In [ ]:
def plot_rocs(scores_and_names, y):
    pack=[] 
    for s, n in scores_and_names: 
        fpr, tpr, thresholds = roc_curve(y.ravel(), s)
        pack.append([n, fpr,tpr,thresholds])

    plt.figure()
    lw=2
    for n, fpr, tpr, thresholds in pack:
        plt.plot(fpr, tpr, lw=lw, label="%s (AUC = %0.2f)" % (n, auc(fpr, tpr))) 

    plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver Operating Characteristic curve")
    plt.legend(loc="lower right")
    plt.show()

    
def decision_function(myX, myY, var, val):
    #pred=predict(myX, var, val)
    pred=myX[var]
    #pred=(pred-pred.min())/(pred.max()-pred.min())
    return -pred

y_score = fitted_bdt_ada.decision_function(X_test)
plot_rocs([ [fitted_bdt_ada.decision_function(X_test), 'AdaBoost'],
            [decision_function(X_test, y_test, tested_var, threshold_val), 'Personalised score']],
          y_test)